# Word Count Example

Spark (as Hadoop) is a massively parallel system for counting words. Although DataFrames are not the perfect data structure for implementing a word count, it is still possible.

In [1]:
from pyspark.sql.functions import *

In [2]:
# Load text "Alice in wonderland"
text = spark.read.text("s3://dimajix-training/data/alice")

In [ ]:
# Inspect Schema of "text" DataFrame
text.printSchema()

In [3]:
# Print first 10 entries of "text" DataFrame
text.limit(10).toPandas()

,value
0,Project Gutenberg's Alice's Adventures in Wond...
1,
2,This eBook is for the use of anyone anywhere a...
3,almost no restrictions whatsoever. You may co...
4,re-use it under the terms of the Project Guten...
5,with this eBook or online at www.gutenberg.org
6,
7,
8,Title: Alice's Adventures in Wonderland
9,


## Extracting words
Now we a DataFrame with a single column. Each entry contains a line of the original text file. We need to extract the individual words in three steps:
1. Split each line into words using the `split` function. This will result in a DataFrame with a single column, which contains a list of words
2. Convert each list of words into individual records using the `explode` function
3. Remove empty words using an appropriate `filter` expression

In [7]:
# 1. Using the split function, split each record into a list of words
word_lists = text.select(split(text.value, ' ').alias("word_list"))
# 2. Using the explode function, convert each list into individual records
words = word_lists.select(explode(word_lists.word_list).alias("word"))
# 3. Remove empty words
non_empty_words = words.filter(words.word != '')
# Show first 10 entries
non_empty_words.limit(10).toPandas()

,word
0,Project
1,Gutenberg's
2,Alice's
3,Adventures
4,in
5,"Wonderland,"
6,by
7,Lewis
8,Carroll
9,This


## Counting Words
Now that we have a DataFrame containing an individual word per record, we can count word frequencies using grouping and aggregation.
1. Group by word
2. Count the size of each group
3. Sort by frequency (descinding)

In [8]:
# 1. & 2. Group by "word" column and count the size of each group
result = non_empty_words.groupBy(words.word).count()

# 3. Sort words by frequency (descending)
sorted_result = result.orderBy(result['count'].desc())

# Print first 10 entries (most frequent words)
sorted_result.limit(10).toPandas()

,word,count
0,the,1664
1,and,780
2,to,773
3,a,662
4,of,596
5,she,484
6,said,416
7,in,401
8,it,356
9,was,329
